<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/TEST/thirteen_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thirteen Test - Test our processor, CUDA nedded?

Let's see if our processor is good.

## Setup

The following cells set up the environment:
1. Clone the XHUMAN repository
2. Install dependencies
3. Authenticate with HuggingFace Hub
4. Import required libraries

In [1]:
!rm -rf XHUMAN/

In [2]:
# Clone

Cloning into 'XHUMAN'...
remote: Enumerating objects: 2295, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 2295 (delta 52), reused 55 (delta 31), pack-reused 2212 (from 1)
Receiving objects: 100% (2295/2295), 7.73 MiB | 36.48 MiB/s, done.
Resolving deltas: 100% (1547/1547), done.


In [3]:
%cd XHUMAN

/content/XHUMAN


In [4]:
!uv pip install -e .[pi]

Using Python 3.12.12 environment at: /usr
Resolved 215 packages in 1.57s
Prepared 1 package in 1.10s
Uninstalled 1 package in 1ms
Installed 1 package in 1ms
 ~ xhuman==0.1.0 (from file:///content/XHUMAN)


In [5]:
import time
from contextlib import nullcontext
from typing import Any

import torch
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs
from torch.optim import Optimizer

from lerobot.configs import parser
from lerobot.datasets.sampler import EpisodeAwareSampler
from lerobot.datasets.utils import cycle
from lerobot.optim.factory import make_optimizer_and_scheduler
from lerobot.policies.pretrained import PreTrainedPolicy
from lerobot.utils.logging_utils import AverageMeter, MetricsTracker
from lerobot.utils.random_utils import set_seed
from lerobot.utils.train_utils import load_training_state
from lerobot.utils.utils import (
    format_big_number,
    has_method,
    init_logging,
)

from xhuman.policies.factory import make_xhuman_policy, make_xhuman_pre_post_processors
from xhuman.configs.train import TrainPipelineConfigXHUMAN
from xhuman.datasets.factory import make_dataset_xhuman
from xhuman.datasets.utils import split_train_eval_episodes
from xhuman.logger import logger

## Helper Functions

These functions handle dataset loading and policy updates. They are designed to work with distributed training using HuggingFace Accelerate.

In [6]:
def load_dataset(cfg: TrainPipelineConfigXHUMAN, episodes: list[int], is_main_process: bool = True, accelerator: Accelerator | None = None):
    """
    Load the dataset for training and evaluation.
    """
    # Dataset loading synchronization: main process downloads first to avoid race conditions
    cfg.dataset.episodes = episodes
    cfg.dataset.train_with_subtasks = True
    if is_main_process:
        logger.info("Creating dataset")
        dataset = make_dataset_xhuman(cfg)

    accelerator.wait_for_everyone()

    # Now all other processes can safely load the dataset
    if not is_main_process:
        dataset = make_dataset_xhuman(cfg)

    return dataset

In [7]:
from xhuman.policies.pi05ki.configuration_pi05ki import PI05KIConfig

policy_config = PI05KIConfig(repo_id="none",device="cuda")

[23:10:54] WARNING  No accelerated backend detected. Using default cpu, this will be slow.              ]8;id=270103;file:///usr/local/lib/python3.12/dist-packages/lerobot/utils/utils.py\utils.py]8;;\:]8;id=461272;file:///usr/local/lib/python3.12/dist-packages/lerobot/utils/utils.py#52\52]8;;\

           WARNING  Device 'cuda' is not available. Switching to 'cpu'.                              ]8;id=309875;file:///usr/local/lib/python3.12/dist-packages/lerobot/configs/policies.py\policies.py]8;;\:]8;id=659981;file:///usr/local/lib/python3.12/dist-packages/lerobot/configs/policies.py#86\86]8;;\

## Configuration and Setup

Configure your dataset and policy settings here. The dataset configuration specifies which HuggingFace repository to load, and the policy configuration sets up the PI05 model architecture.

In [8]:
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id="NONHUMAN-RESEARCH/test-general-idx",
)

In [9]:

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config # Example policy configuration, replace with your actual policy path
)
cfg.validate()

## Training Setup

Initialize the Accelerator for distributed training and set up the training environment. The accelerator automatically handles:
- Multi-GPU training
- Mixed precision training
- Gradient synchronization across processes

In [10]:
# Create Accelerator
# It will automatically detect if running in distributed mode or single-process mode
# We set step_scheduler_with_optimizer=False to prevent accelerate from adjusting the lr_scheduler steps based on the num_processes
# We set find_unused_parameters=True to handle models with conditional computation
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(step_scheduler_with_optimizer=False, kwargs_handlers=[ddp_kwargs])

init_logging(accelerator=accelerator)

# Determine if this is the main process (for logging and checkpointing)
is_main_process = accelerator.is_main_process

# Set seed if specified
if cfg.seed is not None:
    set_seed(cfg.seed, accelerator=accelerator)

# Use accelerator's device
device = accelerator.device
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

In [13]:
import sys
import importlib

# Polyfill 'imp' for Python 3.12 compatibility
if "imp" not in sys.modules:
    import types
    imp = types.ModuleType("imp")
    imp.reload = importlib.reload
    sys.modules["imp"] = imp

%load_ext autoreload
%autoreload 2

In [60]:
import lerobot.datasets.lerobot_dataset
from lerobot.datasets.utils import load_nested_dataset as original_load_nested_dataset
import datasets

# Monkey patch to handle schema mismatch
# The dataset parquet files have a schema that conflicts with the strict fixed-length definition
# expected by the code. We patch the loader to ignore strict features if loading fails.
def patched_load_nested_dataset(pq_dir, features=None, episodes=None):
    try:
        return original_load_nested_dataset(pq_dir, features=features, episodes=episodes)
    except (datasets.builder.DatasetGenerationError, datasets.table.CastError, Exception) as e:
        # Fallback if strict schema casting fails
        print(f"Warning: Dataset loading failed with strict features ({type(e).__name__}). Retrying without features constraint.")
        return original_load_nested_dataset(pq_dir, features=None, episodes=episodes)

lerobot.datasets.lerobot_dataset.load_nested_dataset = patched_load_nested_dataset

DEBUG_MODE = True  # Set to False for full training
DEBUG_MAX_EPISODES = 3  # Use only first N episodes for debugging

# First, get total episodes count (load minimal dataset to check)
if is_main_process:
    temp_dataset = make_dataset_xhuman(cfg)
    total_episodes = temp_dataset.meta.total_episodes
    del temp_dataset
    logger.info(f"Total episodes available: {total_episodes}")
else:
    # For non-main processes, use a reasonable default
    # In practice, this will be synced after main process loads
    total_episodes = 4  # Fallback - adjust if needed

accelerator.wait_for_everyone()

# Limit episodes for debugging
if DEBUG_MODE:
    episodes = list(range(min(DEBUG_MAX_EPISODES, total_episodes)))
    if is_main_process:
        logger.info(f"DEBUG MODE: Using only {len(episodes)} episodes")
else:
    episodes = list(range(total_episodes))

# Split episodes
train_episodes, eval_episodes = split_train_eval_episodes(
    episodes, split_ratio=cfg.split_ratio, seed=42
)

# Load dataset with ONLY train episodes (proper way to filter)
# This uses the load_dataset helper function which sets cfg.dataset.episodes
if is_main_process:
    logger.info(f"Loading train dataset with {len(train_episodes)} episodes")
dataset = load_dataset(cfg, train_episodes, is_main_process=is_main_process, accelerator=accelerator)
dataset.train_with_subtasks = True


                                                    task_index
Pick the fruits from the table and place them i...           0


[23:28:36] INFO     Total episodes available: 437                                    ]8;id=865708;file:///tmp/ipython-input-2723415767.py\ipython-input-2723415767.py]8;;\:]8;id=229021;file:///tmp/ipython-input-2723415767.py#26\26]8;;\

           INFO     DEBUG MODE: Using only 3 episodes                                ]8;id=624756;file:///tmp/ipython-input-2723415767.py\ipython-input-2723415767.py]8;;\:]8;id=915548;file:///tmp/ipython-input-2723415767.py#38\38]8;;\

           INFO     Loading train dataset with 2 episodes                            ]8;id=234053;file:///tmp/ipython-input-2723415767.py\ipython-input-2723415767.py]8;;\:]8;id=146316;file:///tmp/ipython-input-2723415767.py#50\50]8;;\

           INFO     Creating dataset                                                   ]8;id=935518;file:///tmp/ipython-input-711719669.py\ipython-input-711719669.py]8;;\:]8;id=571858;file:///tmp/ipython-input-711719669.py#9\9]8;;\

                                                    task_index
Pick the fruits from the table and place them i...           0


## Dataset and Model Information

Display metadata about the loaded dataset and model. This includes:
- Total number of episodes and frames
- Model parameter counts
- Effective batch size (accounting for distributed training)

In [62]:
# Display dataset metadata and model configuration
if is_main_process:
    from pprint import pprint

    print("=" * 80)
    print("DATASET METADATA")
    print("=" * 80)
    print(f"\nDataset Repository: {dataset.repo_id}")
    print(f"Total Episodes: {dataset.meta.total_episodes}")
    print(f"Training Episodes: {len(train_episodes)}")
    print(f"Number of Frames: {dataset.num_frames:,}")
    print(f"Number of Episodes (loaded): {dataset.num_episodes}")


    print("\n" + "=" * 80)

DATASET METADATA

Dataset Repository: NONHUMAN-RESEARCH/test-general-idx
Total Episodes: 437
Training Episodes: 2
Number of Frames: 2,251
Number of Episodes (loaded): 2



In [63]:
# Create processors
preprocessor, postprocessor = make_xhuman_pre_post_processors(
    policy_cfg=cfg.policy,
    pretrained_path=cfg.policy.pretrained_path,
)

In [64]:
# Print training info
if is_main_process:
    logger.info(f"Output dir: {cfg.output_dir}")
    logger.info(f"Steps: {cfg.steps} ({format_big_number(cfg.steps)})")
    logger.info(f"Dataset frames: {dataset.num_frames} ({format_big_number(dataset.num_frames)})")
    logger.info(f"Dataset episodes: {dataset.num_episodes}")
    num_processes = accelerator.num_processes
    effective_bs = cfg.batch_size * num_processes
    logger.info(f"Effective batch size: {cfg.batch_size} x {num_processes} = {effective_bs}")

[23:28:51] INFO     Output dir: outputs/train/2026-01-22/23-10-54_pi05_ki             ]8;id=31244;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=98246;file:///tmp/ipython-input-1805760154.py#3\3]8;;\

           INFO     Steps: 100000 (100K)                                              ]8;id=27824;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=588508;file:///tmp/ipython-input-1805760154.py#4\4]8;;\

           INFO     Dataset frames: 2251 (2K)                                         ]8;id=571412;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=439898;file:///tmp/ipython-input-1805760154.py#5\5]8;;\

           INFO     Dataset episodes: 2                                               ]8;id=848749;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=911527;file:///tmp/ipython-input-1805760154.py#6\6]8;;\

           INFO     Effective batch size: 8 x 1 = 8                                   ]8;id=732052;file:///tmp/ipython-input-1805760154.py\ipython-input-1805760154.py]8;;\:]8;id=443143;file:///tmp/ipython-input-1805760154.py#9\9]8;;\

In [65]:
# Create dataloader
if hasattr(cfg.policy, "drop_n_last_frames"):
    logger.info(f"Dropping {cfg.policy.drop_n_last_frames} last frames")
    shuffle = False
    sampler = EpisodeAwareSampler(
        dataset.meta.episodes["dataset_from_index"],
        dataset.meta.episodes["dataset_to_index"],
        drop_n_last_frames=cfg.policy.drop_n_last_frames,
        shuffle=True,
    )
else:
    logger.info("Not dropping any frames")
    shuffle = True
    sampler = None

           INFO     Not dropping any frames                                           ]8;id=800581;file:///tmp/ipython-input-988475408.py\ipython-input-988475408.py]8;;\:]8;id=352944;file:///tmp/ipython-input-988475408.py#12\12]8;;\

In [66]:
# Training initialization
# This logs the start of training and shows how many episodes will be used
if is_main_process:
    logger.info("Start offline training on a fixed dataset")
    logger.info(f"Train episodes: {len(train_episodes)}")
    logger.info(f"Total training steps: {cfg.steps}")

[23:28:53] INFO     Start offline training on a fixed dataset                         ]8;id=376417;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=888662;file:///tmp/ipython-input-2557974599.py#4\4]8;;\

           INFO     Train episodes: 2                                                 ]8;id=45561;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=765179;file:///tmp/ipython-input-2557974599.py#5\5]8;;\

           INFO     Total training steps: 100000                                      ]8;id=396922;file:///tmp/ipython-input-2557974599.py\ipython-input-2557974599.py]8;;\:]8;id=82627;file:///tmp/ipython-input-2557974599.py#6\6]8;;\

In [67]:
from torch.utils.data import Dataset

class SmartSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    def __getattr__(self, name):
        # This is the magic part:
        # If the code asks for 'meta', 'fps', etc., and this class doesn't have it,
        # it automatically looks inside the original dataset.
        return getattr(self.dataset, name)

# --- USAGE ---
# Use SmartSubset instead of torch.utils.data.Subset
debug_subset = SmartSubset(dataset, range(0, 50))

# Now create your loader normally
train_dataloader = torch.utils.data.DataLoader(
    debug_subset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [23]:
train_dataloader

In [68]:
dl_iter = iter(train_dataloader)

In [ ]:
import sys
import importlib

# Polyfill 'imp' for Python 3.12 compatibility
if "imp" not in sys.modules:
    import types
    imp = types.ModuleType("imp")
    imp.reload = importlib.reload
    sys.modules["imp"] = imp

%load_ext autoreload
%autoreload 2

# Call Subtask Processor
Important track the index for loss mask

In [72]:
frames = next(dl_iter)

[23:31:32] INFO     Loaded item with index 3                                                  ]8;id=183573;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py\xhuman_dataset.py]8;;\:]8;id=82804;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py#246\246]8;;\

           INFO     Loaded item with index 18                                                 ]8;id=716318;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py\xhuman_dataset.py]8;;\:]8;id=252528;file:///content/XHUMAN/xhuman/datasets/xhuman_dataset.py#246\246]8;;\

In [73]:
frames.keys()

dict_keys(['observation.images.left', 'observation.images.top', 'observation.images.right', 'action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index', 'general_task_index', 'action_is_pad', 'task', 'general_task', 'train_with_subtask'])

In [74]:
frames["task"]

['pick up the strawberry and put it in the basket',
 'pick up the strawberry and put it in the basket']

In [75]:
frames["general_task_index"]

tensor([0, 0])

In [76]:
frames["general_task"]

['Pick the fruits from the table and place them in the basket',
 'Pick the fruits from the table and place them in the basket']

In [ ]:
# frames["subtask"] = frames["task"]

In [77]:
batch = preprocessor.subtask(frames)

In [78]:
batch.keys()

dict_keys(['action', 'next.reward', 'next.done', 'next.truncated', 'info', 'action_is_pad', 'task', 'index', 'task_index', 'episode_index', 'train_with_subtask', 'subtask', 'subtask_tokens', 'observation.images.left', 'observation.images.top', 'observation.images.right', 'observation.state', 'observation.language.tokens', 'observation.language.attention_mask'])

In [79]:
batch["task"]

['Task: pick the fruits from the table and place them in the basket. Subtask: ',
 'Task: pick the fruits from the table and place them in the basket. Subtask: ']

In [87]:
sub_tokens = batch["subtask_tokens"]

In [ ]:
tokens  = batch["observation.language.tokens"]
tokens

In [ ]:
sub_lan = tokenize.batch_decode(sub_tokens)
sub_lan

In [ ]:
tokenslang = tokenize.batch_decode(tokens,skip_special_tokens=True)
tokenslang


['Task: pick up the strawberry and put it in the basket. Subtask: ',
 'Task: pick up the strawberry and put it in the basket. Subtask: ']

# Test loss calculation

In [ ]:
tokens_id = "observation.language.tokens"
mask_id = "observation.language.attention_mask"

tokens, masks = batch[tokens_id], batch[mask_id]
noise = None
time = None
# Processor should have a method to get the subtask tokenized
# List of tensors


In [84]:
from transformers import AutoTokenizer

tokenize = AutoTokenizer.from_pretrained("google/paligemma-3b-pt-224")


In [32]:
import pandas as pd
import os

name1 = "general_tasks.parquet"
name2 = "tasks.parquet"

# Visualize the first rows , I want that you read the first rows from those files, they are on /content

file1_path = os.path.join("/content", name1)
file2_path = os.path.join("/content", name2)

print(f"Reading {file1_path}")
general_tasks_df = pd.read_parquet(file1_path)
display(general_tasks_df)



Reading /content/general_tasks.parquet


,task_index
Pick the fruits from the table and place them in the basket,0


In [34]:
print(f"Reading {file2_path}")
tasks_df = pd.read_parquet(file2_path)
display(tasks_df)

Reading /content/tasks.parquet


,task_index
pick up the strawberry and put it in the basket,0
pick up the green pear and put it in the basket,1
pick up the banana and put it in the basket,2
pick up the grapes and put it in the basket,3
